# Simulation study

In [ ]:
import numpy as np
from scipy.stats import binom, norm, t
from methods import birge, random_effects_hksj, binomial_method, binomial_adapt, binomial_sigmacdf, random_effects_dl, sign_rank, flip_interval, random_effects_mle, vniim
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt
import pandas as pd

# use tex
plt.rc("text", usetex=True)

## Table 1

In [ ]:
# | label: table-n-nb

p = 0.5
coverages_mu = []
target_coverage = 0.6827
tail_alpha = (1 - target_coverage) / 2



# ns = [3, 10, 31, 100, 316, 1000]
# ns = np.logspace(0.5, 3, 6).astype(int)
#ns = np.logspace(0.5, 2, 4).astype(int)
ns = [3, 10, 31]
tail_alphas = {}
target_coverages = {}
rows = []
for n in ns:
    ks = np.arange(0, n + 1)
    cdf = binom.cdf(ks, n, p)
    tail_alpha_achieved = cdf[np.argmax(np.array(cdf) >= tail_alpha) - 1]
    tail_alphas[n] = tail_alpha_achieved
    target_coverages[n] = 1 - 2 * tail_alpha_achieved
    z_score = norm.ppf(1 - tail_alpha_achieved)
    new_row = {
        "n": n,
        "target_coverage": np.round(target_coverages[n], 3),
        "tail_alpha": np.round(tail_alpha_achieved, 3),
        "z_score": np.round(z_score, 3),
    }
    rows.append(new_row)

table = pd.DataFrame(rows)
# set n column as index
table.set_index("n", inplace=True)
# print as markdown table
print(table.to_markdown())

## Simulation code

In [ ]:
# mus = np.logspace(-3, 3, 13)
mus = np.logspace(-2, 2, 9)

models = ['nonoise', 're_hksj', 're_dl', 'birge', 'signrank', 're_mle', 'pdg', 'vniim'] #'flip']#, 'binomial_adapt', 'binomial_sigmacdf']

settings = [
    "random_effects",
    "birge",
    "random_effects_outliers",
    # "adversarial",
    # "random_effects_corr",
]

results = {}
for i in range(len(settings)):
    setting = settings[i]
    results[setting] = {}
    total = len(ns) * len(mus)
    for n, mu in tqdm(itertools.product(ns, mus), total=total):
        results[setting][(n, mu)] = {}

        tail_alpha = tail_alphas[n]
        target_coverage = 1 - 2 * tail_alpha
        z_alpha = norm.ppf(1 - tail_alpha)
        t_alpha = t.ppf(1 - tail_alpha, n - 1)

        ks = np.arange(0, n + 1)
        cdf = binom.cdf(ks, n, p)

        coverages = {k: [] for k in models}
        coverages_ba = []
        coverages_bs = []
        lengths = {k : [] for k in models}
        midpoints = {k : [] for k in models}

        for j in range(1000):
            # noise_magnitudes = np.clip(np.random.normal(1, 0.25, n), 0.1, None)
            noise_magnitudes = np.random.exponential(1, n)
            if setting == "random_effects":
                systematic_errors = np.random.normal(0, mu, n)
                random_errors = np.random.normal(0, noise_magnitudes, n)
                values = systematic_errors + random_errors
            elif setting == "birge":
                random_errors = np.random.normal(0, noise_magnitudes, n)
                values = random_errors * mu
            elif setting == "random_effects_outliers":
                systematic_errors = np.random.standard_cauchy(n) * mu
                random_errors = np.random.normal(0, noise_magnitudes, n)
                values = systematic_errors + random_errors
            elif setting == "adversarial":
                random_errors = np.random.normal(0, noise_magnitudes, n)
                values = mu + random_errors
            elif setting == "random_effects_corr":
                if n > 100:
                    continue
                Sigma = mu**2 * (0.8 * np.eye(n) + 0.2 * np.ones((n, n)))
                rng = np.random.default_rng()
                systematic_errors = rng.multivariate_normal(
                    np.zeros(n), Sigma, method="cholesky"
                )
                # systematic_errors = np.random.multivariate_normal(np.zeros(n), Sigma) TOO SLOW
                # noise_magnitudes = np.random.exponential(1, n)
                random_errors = np.random.normal(0, noise_magnitudes, n)
                values = systematic_errors + random_errors
            else:
                raise ValueError("setting not recognized")

            values_sort = np.sort(values)

            lower_nonoise, _ = binomial_method(
                values_sort, p=p, target=tail_alpha, which="lower", cdf=cdf
            )
            upper_nonoise, _ = binomial_method(
                values_sort, p=p, target=tail_alpha, which="upper", cdf=cdf
            )

            interval_nonoise = [lower_nonoise, upper_nonoise]
            covers_nonoise = interval_nonoise[0] < 0 and interval_nonoise[1] > 0

            interval_signrank, _ = sign_rank(values, coverage=target_coverage)


            # interval_binomial_adapt, coverage_ba = binomial_adapt(values, noise_magnitudes, p, 0.6827, cdf, which='random')
            # coverages_ba.append(coverage_ba)
            # interval_binomial_sigmacdf, cvg = binomial_sigmacdf(values, noise_magnitudes, p, 0.6827)
            # coverages_bs.append(cvg)

            # calculate using random-effects model
            interval_re_hksj, muhat, sigma, _ = random_effects_hksj(
                values, noise_magnitudes, talpha=t_alpha
            )

            interval_re_dl, muhat, sigma, _ = random_effects_dl(
                values, noise_magnitudes, zalpha=z_alpha
            )

            interval_re_mle, muhat, sigma, _ = random_effects_mle(
                values, noise_magnitudes, zalpha=z_alpha
            )

            interval_birge, muhat, sigma, _ = birge(
                values, noise_magnitudes, zalpha=z_alpha
            )

            interval_pdg, muhat, sigma, _ = birge(
                values, noise_magnitudes, zalpha=z_alpha, pdg=True
            )

            interval_vniim, muhat = vniim(
                values, noise_magnitudes, zalpha=z_alpha
            )

            # interval_flip, _ = flip_interval(values, coverage=target_coverage, mode='median', boot=False)

            intervals = {
                "nonoise": interval_nonoise,
                "re_hksj": interval_re_hksj,
                "re_dl": interval_re_dl,
                "re_mle": interval_re_mle,
                "birge": interval_birge,
                "signrank": interval_signrank,
                "pdg": interval_pdg,
                "vniim": interval_vniim,
                # "flip": interval_flip,
                # "binomial_adapt": interval_binomial_adapt,
                # "binomial_sigmacdf": interval_binomial_sigmacdf
            }

            for k, v in intervals.items():
                coverages[k].append(v[0] < 0 and v[1] > 0)
                lengths[k].append(v[1] - v[0])
                midpoints[k].append((v[0] + v[1]) / 2)

        result_dict = results[setting][(n, mu)]
        # result_dict['t_coverage_binomial_adapt'] = np.mean(coverages_ba)
        # result_dict['t_coverage_binomial_sigmacdf'] = np.mean(coverages_bs)

        for k in models:
            lengths[k] = np.array(lengths[k])

            result_dict["coverage_" + k] = np.mean(coverages[k])
            result_dict["length_median_" + k] = np.median(lengths[k])
            result_dict["length_mean_" + k] = np.mean(lengths[k])
            result_dict["midpoint_median" + k] = np.median(np.abs(midpoints[k]))
            result_dict["midpoint_mean_" + k] = np.mean(np.abs(midpoints[k]))
            result_dict["length_median_relbinomial_" + k] = np.median(lengths[k]/lengths["nonoise"])

In [ ]:
results['random_effects'][(3, 1)]

## Simulation figures

In [ ]:
colors = {
    "nonoise": "red",
    "re_hksj": "blue",
    "re_dl": "grey",
    "re_mle": "purple",
    "birge": "green",
    "signrank": "orange",
    "pdg": "black",
    "vniim": "yellow"
    # "flip": "brown",
    # "binomial_adapt": "green",
    # "binomial_sigmacdf": "green"
    }
friendly_labels = {
    "nonoise": "Binomial",
    "re_hksj": "HKSJ",
    # "re_dl": "DL",
    "re_mle": "MLE",
    "birge": "Birge Ratio",
    # "signrank": "Sign Rank",
    "pdg": "PDG",
    "vniim": "VNIIM",
    # "binomial_adapt": "Binomial Adapt (BA)"
    # 'binomial_sigmacdf': 'Binomial SigmaCDF (BS)'
}
methods = friendly_labels.keys()
setting_labels = {
    "random_effects": "Random Effects",
    "birge": "Birge",
    "random_effects_outliers": "Random Effects with Cauchy",
    "adversarial": "Offset",
    "random_effects_corr": "Correlated Random Effects",
}
print(target_coverages)
for setting in settings:
    fig, axs = plt.subplots(2, 1, figsize=(4, 6), sharex=True)
    axs[0].axhline(0, color="black", lw=1, ls="--")
    for method in methods:
        # if method == 'birge':
        #     continue
        print(method)
        for i, n in enumerate(ns):
            if results[setting][(n, mus[0])] == {}:
                continue
            alpha = (i + 1) / len(ns)
            target_coverage = target_coverages[n]
            if method == 'binomial_adapt':
                target_coverage = np.array([results[setting][(n, mu)]['t_coverage_binomial_adapt'] for mu in mus])
            if method == 'binomial_sigmacdf':
                target_coverage = np.array([results[setting][(n, mu)]['t_coverage_binomial_sigmacdf'] for mu in mus])
            data = [results[setting][(n, mu)][f"coverage_{method}"] for mu in mus]
            data = np.array(data) - target_coverage
            if i == len(ns) - 1:
                label = friendly_labels[method]
            else:
                label = None
            axs[0].plot(mus, data, color=colors[method], label=label, alpha=alpha)
            # data = np.array([results[setting][(n, mu)][f"length_median_{method}"] for mu in mus])
            # data_bin = np.array([results[setting][(n, mu)][f"length_median_nonoise"] for mu in mus])
            # axs[1].plot(mus, data/data_bin, color=colors[method], label=label, alpha=alpha)
            data = np.array([results[setting][(n, mu)][f"length_median_relbinomial_{method}"] for mu in mus])
            axs[1].plot(mus, data, color=colors[method], label=label, alpha=alpha)
            # data = np.array([results[setting][(n, mu)][f"midpoint_{method}"] for mu in mus])
            # data_bin = np.array([results[setting][(n, mu)][f"midpoint_nonoise"] for mu in mus])
            # axs[2].plot(mus, data/data_bin, color=colors[method], label=label, alpha=alpha)
            # axs[1].plot(mus, data, color=colors[method], label=label, alpha=alpha)
    # add top and right ticks
    for ax in axs:
        ax.yaxis.set_ticks_position("both")
        ax.xaxis.set_ticks_position("both")
        # ax.tick_params(direction='in')
        ax.tick_params(which="both", direction="in")
        ax.set_xscale("log")
    axs[1].set_xlabel(r"$\tau$: size of systematic error relative to noise")
    # axs[0].set_ylim(-0.13, 0.075)

    # log y scale on right axis
    axs[1].set_yscale("log")
    # axs[2].set_yscale("log")
    # axs[1].set_ylim(0, None)

    # add legend with colors and corresponding labels
    # plt.legend(frameon=False)

    axs[0].set_ylabel("Coverage probability $-$ target coverage")
    axs[1].set_ylabel("Median interval width relative to Binomial")
    # axs[2].set_ylabel(r"Median distance of midpoint from $\theta$")
    plt.suptitle(f"{setting_labels[setting]}")
    # plt.savefig(f"figs/performance_{setting}.pdf", bbox_inches="tight")
    plt.tight_layout()
    plt.savefig(f'figs/performance_{setting}_exp.png', bbox_inches="tight", dpi=300)
    plt.show()

In [ ]:
from methods import binomial_adapt
ys = np.arange(0, 15, 1) * 0.1
n = len(ys)
sigmas = np.ones(len(ys))

probs = binomial_adapt(ys, sigmas, p=0.5)

# first element in each row where the cumulative sum is greater than 0.6827
idx_row = np.argmax(probs >= 0.6827, axis=1)

idx_col = n - np.argmax(probs[::-1,:] >= 0.6827, axis=0) - 1

# array elements which correspond to both an element of idx_row and idx_col
selector_row = np.array([idx_row, np.arange(n)]).T
selector_col = np.array([np.arange(n), idx_col]).T

selected_row = np.zeros((n, n), dtype=bool)
selected_col = np.zeros((n, n), dtype=bool)

selected_row[np.arange(n), idx_row] = True
selected_col[idx_col, np.arange(n)] = True
plt.imshow((selected_row.astype(int) + selected_col.astype(int)) * (probs>0.6827).astype(int))
plt.ylabel('Index of lower bound')
plt.xlabel('Index of upper bound')
plt.show()

In [ ]:
interval_good = selected_row & selected_col & (probs>0.6827)
# indices of trues
intervals_idx = np.array(interval_good.nonzero()).T
intervals = ys[intervals_idx]
intervals
interval_lengths = intervals[:,1] - intervals[:,0]
interval_lengths
interval_probs = probs[intervals_idx[:,0], intervals_idx[:,1]]
interval_probs

In [ ]:
errors_l = np.abs((ys[:,np.newaxis] - intervals[:,0]))/sigmas[:,np.newaxis]
errors_u = np.abs((ys[:,np.newaxis] - intervals[:,1]))/sigmas[:,np.newaxis]
within = (ys[:, np.newaxis] >= intervals[:,0]) & (ys[:, np.newaxis] <= intervals[:,1])
dists = np.minimum(errors_l, errors_u) * (~within)
total_dist = np.sum(dists, axis=0)
total_dist

In [ ]:
within

In [ ]:
print(interval_lengths)

In [ ]:
plt.imshow(selected_col)

In [ ]:
plt.imshow(selected_row & selected_col)

In [ ]:
np.max(probs[selected_row & selected_col])

In [ ]:
selected = (selected_row | selected_col) & (probs >= 0.6827)

In [ ]:
selected[:30, 70:].sum()

In [ ]:
selected

In [ ]:
print(np.sum(selected))

In [ ]:
probs[idx_row, np.arange(n)]

In [ ]:

selector_row

In [ ]:
import numpy as np
import Rmath4
from tqdm import tqdm
conf_level = 0.6827
alpha = 1-conf_level
covers = []
for i in tqdm(range(8000000)):
    y = np.random.normal(5,2,5)
    n = len(y)
    w = np.add.outer(y, y)/2
    w = np.sort(w[np.tril_indices(w.shape[0], 0)])
    qu = int(Rmath4.qsignrank(alpha/2, n, 0, 0))
    if qu == 0:
        qu = 1
    ql = int(n*(n+1) / 2 - qu)
    achieved_alpha = 2*Rmath4.psignrank(qu-1, n, 0, 0)
    # print(achieved, len(w), qu)
    # lower = w[ql+1-1]
    # upper = w[qu-1]
    lower = w[ql+1-1]
    upper = w[qu-1]
    # print(lower, upper)
    covers.append(lower < 5 and upper > 5)
print(np.mean(covers), 1-achieved_alpha)

In [ ]:
print(lower, upper)

In [ ]:
Rmath4.qsignrank

In [ ]:
(np.tril(w) != 0).sum()